In [13]:
import nltk

import string

import stanfordnlp  

from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'C:\Users\spike\python\stanford-corenlp-full-2018-10-05',lang='en')

def fragsT(s): #定位了各类括号，找到句子中的各种结构
    parse=nlp.parse(s) #.replace(' ','')
    parsepos=""
    frags=[]
    #print(parse)
    cln=[]
    crn=[]
    x1=0
    #x2=0
    #y1=0
    y2=0
    x0=0
    for n1 in range(0,len(parse)):
        if parse[n1]=='(':
            t=0
            t0=0
            while n1-t0-1>=0 and parse[n1-1-t0]==' ':
                t0+=1
            if parse[n1-1-t0]=='\n' :
                t=t0
            else:
                t=0
            x1+=1
            y1=0
            for cha in parse[n1:]:
                if cha==")":
                    y1+=1
            cln.append(((n1,x1-1,y1),t))
        elif parse[n1]==")":
            x2=x1
            y2=0
            for cha in parse[n1:]:
                if cha==")":
                    y2+=1
            crn.append((n1,x1,y2-1))
    #print((cln,crn))
    #print(len(cln),len(crn))
    for cl in cln:
        frag=""
        n=0
        while frag=="" and n<=len(crn):
                cr=crn[n]
                if cl[0][0]<cr[0] and  cl[0][1]-cr[2]==cr[1]-cl[0][2]:
                    #print(((cl,cr)))
                    frag=parse[cl[0][0]:cr[0]+1]
                    frags.append((frag,cl[1]))
                n+=1
    result=[frags[0]]
    for n in range(1,len(frags)):
        n1=n
        while n1>=0 and frags[n1][1]==0:
            n1=n1-1
        result.append((frags[n][0],frags[n1][1]))
    return result

def SPCanalyseText(text): #文本向量化 返回值[0] 短语数量、[1]短语复杂度[2]短语长度 [3]短语成分
    sentlist=readsenttext(text)
    sentdatas=[]
    for s in sentlist:
        sentdatas.append(SPCanalyse(s))  #SPCanaluse=[NP,[NP,NP,NP],['LOVE','LOVE','LOVE']]
    print(sentdatas)
    phrasen=0
    dphrasen=0
    phrasel=0
    data=[]
    phrasecate=[]
    for c in sentdatas: #句子合集中的句子数据
        for c1 in c: #句子中的短语数据合集
            if c1[0] not in phrasecate:
                phrasecate.append(c1[0])
                phrasen+=1#构式类别
           
            phrasel+=len(c1[2])        #构式长度
            if  c1[0]=='NP' and len(c1[1])>=1 and c1[1][0]=='NP':
                components=[c1[1]]
                dphrasen+=len(c1[1])  #构式复杂度
            else:
                components=[c1[0]]
                components.extend(c1[1]) #构式结构
                dphrasen+=len(c1[1])+1
            data.append((phrasen,dphrasen,phrasel,components))
    return data

def SPCanalyse(s):  # Synergetic Phrase Components Analyse 返回列表中有成分，成分直属成分，以及成分下的词汇 
    phcons=("ADJP","ADVP","CONJP","FRAG",
            "INTJ","LST","NAC","NP","NX",
            "PP","PRN","PRT","QP","RRC",
            "UCP","VP","WHADJP","WHAVP",'WHADVP',
            "WHNP","WHPP","X")
    postags=("CC","CD","DT","EX","FW",
             "IN","JJ","JJR","JJS","LS",
             "MD","NN","NNS","NNP","NNPS",
             "PDT","POS","PRP","PRP$","RB",
             "RBR","RBS","RP","SYM","TO","UH",
             "VB","VBD","VBG","VBN","VBP","VBZ",
             "WDT","WP","WP$","WRB")
    clstags=["S","SBAR","SBARQ","SINV","SQ"]
    avoidance=string.punctuation+string.whitespace
    target=fragsT(s)
    print(target)
    ddphs=[]
    for line in target:
        line=line[0].replace('\r\n','').replace('(',' ( ').replace(')',' ) ')
        line=line.split()
        #print(line)
        if line[1] in phcons or line[1] in clstags:
            words=[]
            ddph=[]
            for n in range(2,len(line)):
                
                if line[n] in phcons or line[n]=='CC':
                    x=0
                    y=0
                    for n1 in range(0,n):
                        if line[n1]=='(':
                            x+=1
                        if line[n1]==')' or line[n1] in ('S','SBAR'):
                            y+=1
                    if x-y==2:# or x-y==1:
                        #if not line[1]=='NP' and line[n]=='NP':
                        ddph.append(line[n]) 
                #print(line[n])
                if line[n] not in phcons and line[n] not in postags and \
                line[n] not in clstags and line[n] not in avoidance:# or line[n]==',':
                    words.append(line[n])
                if line[n] in avoidance and line[n] not in ('(',')') and line[n-1]==line[n]:
                    words.append(line[n])
                    print(words)
            if line[1]=='VP' and len(line)>=3 and line[3]=='TO':      
                ddphs.append(('TO',ddph,words))
            else:
                ddphs.append((line[1],ddph,words))
            #print(ddphs)
    return ddphs
            

def SCdigText(text,chunk_len=7): #有问题 返回值为 词典 词组：（词块长度，类型）
    avoidance=string.punctuation+string.whitespace
    sentlist=nltk.sent_tokenize(text)
    sentdatas=[]
    for s in sentlist:
        sentdatas.append(SPCanalyse(s))
    target={}
    for s in sentdatas:
        for c in s:
            compo=''
            for w in c[2]:
                if w in avoidance or w in ("'m","'re","n't","'ll"):
                    compo=compo[:-1]
                
                compo+=w+' '
            target[(compo[:-1])]=c[0]
    print(target)
    result={}
    for n in range(0,chunk_len):
        for chunk in target:
            if len(chunk.split())==chunk_len-n:
                result[chunk]=(chunk_len-n,target[chunk])
    return result

In [14]:
s="My name is Adam and I'm a freshman at senior high school."
SCdigText(s)

[("(ROOT\r\n  (S\r\n    (S\r\n      (NP (PRP$ My) (NN name))\r\n      (VP (VBZ is)\r\n        (NP (NNP Adam))))\r\n    (CC and)\r\n    (S\r\n      (NP (PRP I))\r\n      (VP (VBP 'm)\r\n        (NP\r\n          (NP (DT a) (NN freshman))\r\n          (PP (IN at)\r\n            (NP (JJ senior) (JJ high) (NN school))))))\r\n    (. .)))", 0), ("(S\r\n    (S\r\n      (NP (PRP$ My) (NN name))\r\n      (VP (VBZ is)\r\n        (NP (NNP Adam))))\r\n    (CC and)\r\n    (S\r\n      (NP (PRP I))\r\n      (VP (VBP 'm)\r\n        (NP\r\n          (NP (DT a) (NN freshman))\r\n          (PP (IN at)\r\n            (NP (JJ senior) (JJ high) (NN school))))))\r\n    (. .))", 2), ('(S\r\n      (NP (PRP$ My) (NN name))\r\n      (VP (VBZ is)\r\n        (NP (NNP Adam))))', 4), ('(NP (PRP$ My) (NN name))', 6), ('(PRP$ My)', 6), ('(NN name)', 6), ('(VP (VBZ is)\r\n        (NP (NNP Adam)))', 6), ('(VBZ is)', 6), ('(NP (NNP Adam))', 8), ('(NNP Adam)', 8), ('(CC and)', 4), ("(S\r\n      (NP (PRP I))\r\n      (VP (V

{"I'm a freshman at senior high school": (7, 'S'),
 "'m a freshman at senior high school": (7, 'VP'),
 'a freshman at senior high school': (6, 'NP'),
 'My name is Adam': (4, 'S'),
 'at senior high school': (4, 'PP'),
 'senior high school': (3, 'NP'),
 'My name': (2, 'NP'),
 'is Adam': (2, 'VP'),
 'a freshman': (2, 'NP'),
 'Adam': (1, 'NP'),
 'I': (1, 'NP')}